<a href="https://colab.research.google.com/github/kiryu-3/Prmn2023_DS/blob/main/Python/Python_Stats/Stats_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 比率差の検定

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install japanize-matplotlib
import japanize_matplotlib
import seaborn as sns
from scipy import stats
from scipy.stats import norm
from scipy.stats import binom
from scipy.stats import t
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN
from statsmodels.stats.proportion import proportions_ztest

## 母比率の検定

まずは一変数ですが、母比率の仮説検定を行っていきましょう。  
[こちら](https://bit.ly/3k8ioK5)の問題を解いていきます。

### 仮説検定のステップ（再掲）
①帰無仮説と対立仮説を立てる  
②帰無仮説のもとで標本観察を行う  
③帰無仮説を棄却できるかどうかを確認する

#### ① 帰無仮説と対立仮説を立てる  

帰無仮説と対立仮説は以下のようになります。（**両側検定**になります）

帰無仮説$H_0$：「このサイコロの1が出る確率は1/6である」  
対立仮説$H_1$：「このサイコロの1が出る確率は1/6ではない」

#### ② 帰無仮説のもとで標本観察を行う 

比率 $\hat{p} (=\frac{x}{n})$が従う確率分布は二項分布に従います。  
また、$n$が大きいときには正規分布で近似することができます。

平均$μ=p$  
標準偏差$σ=\sqrt{\frac{p(1-p)}{n}}$

母比率$p$とサンプルサイズ$n=12000$を用いて、$z$値を導出します。  
（上方・下方の信頼限界を求める方法もあります）

#### ③ 帰無仮説を棄却できるかどうかを確認する

有意水準を$5$%に設定します。

②で導出した$z$値と、有意水準に基づく$z$値を比較し、  
($p$**値を求めて**) 棄却が必要かどうか決めます。  

（標本統計量が信頼区間の中に入っているかで決めることもあります）

### Pythonによる仮説検定

では母比率の検定を行っていきましょう。

In [2]:
alpha = 0.95
print("信頼係数：{}".format(alpha))
n = 12000
print("標本の⼤きさ：{}".format(n))
p = 2200/12000
print("標本の⽐率：{}".format(p))
μ = 1/6
print("母集団の比率：{}".format(μ))

信頼係数：0.95
標本の⼤きさ：12000
標本の⽐率：0.18333333333333332
母集団の比率：0.16666666666666666


In [3]:
print("H0 : μ＝1/6 [帰無仮説]")  # 1の目が出る確率は1/6
print("H1 : μ≠1/6 [対⽴仮説]")  # 1の目が出る確率は1/6ではない

H0 : μ＝1/6 [帰無仮説]
H1 : μ≠1/6 [対⽴仮説]


In [4]:
# p値を求める方法
ny = float(Decimal(str(norm.ppf(0.975,loc=0,scale=1))).quantize(Decimal('0.001'),
                                                                rounding=ROUND_HALF_UP))
print(f"有意⽔準のz値:±{ny}") # 両側検定なのでt値は2つ

nk = (p-μ)/np.sqrt(μ*(1-μ)/n)  # 検定統計量をz値にする
nk = float(Decimal(str(nk)).quantize(Decimal('0.001'),rounding=ROUND_HALF_UP))
print(f"検定統計量のz値:{nk}")

有意⽔準のz値:±1.96
検定統計量のz値:4.899


In [5]:
if nk<0:
  nyp = float(Decimal(str(norm.cdf(-ny)*100)).quantize(Decimal('0.001'),
                                                       rounding=ROUND_HALF_UP))
  print(f"有意水準：{nyp}%")    # 左側の棄却域の面積
  nkp = float(Decimal(str(norm.cdf(nk)*100)).quantize(Decimal('0.000001'),
                                                      rounding=ROUND_HALF_UP))
  print(f"p値：{nkp}%")
elif nk>=0:
  nyp = float(Decimal(str(norm.sf(ny)*100)).quantize(Decimal('0.001'),
                                                     rounding=ROUND_HALF_UP))
  print(f"有意水準：{nyp}%")    # 右側の棄却域の面積
  nkp = float(Decimal(str(norm.sf(nk)*100)).quantize(Decimal('0.000001'),
                                                     rounding=ROUND_HALF_UP))
  print(f"p値：{nkp}%")

有意水準：2.5%
p値：4.8e-05%


In [6]:
print("p値が有意水準より小さくなったので、帰無仮説を棄却し、対立仮説を採択する")
print("したがって、このサイコロは1の出やすさに関して歪んでいる")

p値が有意水準より小さくなったので、帰無仮説を棄却し、対立仮説を採択する
したがって、このサイコロは1の出やすさに関して歪んでいる


In [7]:
# 信頼区間を求める方法（二項分布）
x1,x2 = binom.interval(alpha,n,μ)  # binomでもnormでも可

print("xの下限値は{}(%)".format(x1/n*100))  # 実際は標本比率が⺟比率を上回ることは明らかなので不要
print("xの上限値は{}(%)".format(x2/n*100))

xの下限値は16.0(%)
xの上限値は17.333333333333336(%)


In [8]:
# 信頼区間を求める方法（正規分布）
xs1,xs2 = norm.interval(alpha,loc=μ,scale=np.sqrt((μ*(1-μ))/n))

print("xの下限値は{}(%)".format(xs1*100))  # 実際は標本比率が⺟比率を上回ることは明らかなので不要
print("xの上限値は{}(%)".format(xs2*100))

xの下限値は15.999873378283983(%)
xの上限値は17.33345995504935(%)


In [9]:
print("棄却域は\n{}(%) > x , x > {}(%) である。\n".format(x1/n*100,x2/n*100))
print("標本比率{}(%)は、棄却域に⼊るため、\n帰無仮説を棄却し、対⽴仮説を採択する".format(p*100))

棄却域は
16.0(%) > x , x > 17.333333333333336(%) である。

標本比率18.333333333333332(%)は、棄却域に⼊るため、
帰無仮説を棄却し、対⽴仮説を採択する


In [10]:
print("したがって、このサイコロは1の出やすさに関して歪んでいる")

したがって、このサイコロは1の出やすさに関して歪んでいる


### 演習

[こちら](https://bit.ly/3YBjZHn)の問題もやってみましょう。

In [11]:
alpha = 0.95
print("信頼係数：{}".format(alpha))
n = 300
print("標本の⼤きさ：{}".format(n))
p = 160/300
print("標本の⽐率：{}".format(p))
μ = 0.5
print("母集団の比率：{}".format(μ))

信頼係数：0.95
標本の⼤きさ：300
標本の⽐率：0.5333333333333333
母集団の比率：0.5


In [12]:
print("H0 : μ＝0.5 [帰無仮説]")  # 「おいしい」と回答する比率は0.5
print("H1 : μ≠0.5 [対⽴仮説]")  # 「おいしい」と回答する比率は0.5ではない

H0 : μ＝0.5 [帰無仮説]
H1 : μ≠0.5 [対⽴仮説]


In [13]:
# p値を求める方法
ny = float(Decimal(str(norm.ppf(0.975,loc=0,scale=1))).quantize(Decimal('0.001'),
                                                                rounding=ROUND_HALF_UP))
print(f"有意⽔準のz値:±{ny}")  # 両側検定なのでt値は2つ

nk = (p-μ)/np.sqrt(μ*(1-μ)/n)  # 検定統計量をz値にする
nk = float(Decimal(str(nk)).quantize(Decimal('0.00001')
,rounding=ROUND_HALF_UP))
print(f"検定統計量のz値:{nk}")

有意⽔準のz値:±1.96
検定統計量のz値:1.1547


In [14]:
if nk<0:
  nyp = float(Decimal(str(norm.cdf(-ny)*100)).quantize(Decimal('0.001'),
                                                       rounding=ROUND_HALF_UP))
  print(f"有意水準：{nyp}%")  # 左側の棄却域の面積
  nkp = float(Decimal(str(norm.cdf(nk)*100)).quantize(Decimal('0.000001'), 
                                                      rounding=ROUND_HALF_UP))
  print(f"p値：{nkp}%")
elif nk>=0:
  nyp = float(Decimal(str(norm.sf(ny)*100)).quantize(Decimal('0.001'),
                                                     rounding=ROUND_HALF_UP))
  print(f"有意水準：{nyp}%")  # 右側の棄却域の面積
  nkp = float(Decimal(str(norm.sf(nk)*100)).quantize(Decimal('0.000001'),
                                                     rounding=ROUND_HALF_UP))
  print(f"p値：{nkp}%")

有意水準：2.5%
p値：12.410665%


In [15]:
print("p値が有意水準より大きくなったので、帰無仮説を棄却せず、対立仮説を採択しない")
print("したがって、「おいしい」と回答する比率は0.5でないとは言えない")

p値が有意水準より大きくなったので、帰無仮説を棄却せず、対立仮説を採択しない
したがって、「おいしい」と回答する比率は0.5でないとは言えない


In [16]:
# 信頼区間を求める方法（二項分布）
x1,x2 = stats.binom.interval(alpha,n,μ)  # binomでもnormでも可

print("xの下限値は{}(%)".format(x1/n*100))  # 実際は標本比率が⺟比率を上回ることは明らかなので不要
print("xの上限値は{}(%)".format(x2/n*100))

xの下限値は44.333333333333336(%)
xの上限値は55.666666666666664(%)


In [17]:
# 信頼区間を求める方法（正規分布）
xs1,xs2 = stats.norm.interval(alpha,loc=μ,scale=np.sqrt((μ*(1-μ))/n))

print("xの下限値は{}(%)".format(xs1*100))  # 実際は標本比率が⺟比率を上回ることは明らかなので不要
print("xの上限値は{}(%)".format(xs2*100))

xの下限値は44.342071329619145(%)
xの上限値は55.65792867038086(%)


In [19]:
print("棄却域は\n{}(%) > x , x > {}(%) である。\n"
.format(x1/n*100,x2/n*100))
print("標本平均{}(%)は、棄却域に入らないため、\n帰無仮説を棄却せず、対立仮説を採択しない".format(p*100))

棄却域は
44.333333333333336(%) > x , x > 55.666666666666664(%) である。

標本平均53.333333333333336(%)は、棄却域に入らないため、
帰無仮説を棄却せず、対立仮説を採択しない


In [20]:
print("したがって、「おいしい」と回答する比率は0.5でないとは言えない")

したがって、「おいしい」と回答する比率は0.5でないとは言えない


## 母比率の差の検定

2つの標本から得た標本比率を使って  
母比率が等しいかを検定していきましょう。

[こちら](https://bit.ly/3k8qPoG)の問題を解いていきます。

### 仮説検定のステップ（再掲）
①帰無仮説と対立仮説を立てる  
②帰無仮説のもとで標本観察を行う  
③帰無仮説を棄却できるかどうかを確認する

#### ① 帰無仮説と対立仮説を立てる  

帰無仮説と対立仮説は以下のようになります。（**両側検定**になります）

帰無仮説$H_0$：「関東地区と関西地区の視聴率は等しい」  
対立仮説$H_1$：「関東地区と関西地区の視聴率は等しくない」

#### ② 帰無仮説のもとで標本観察を行う 

それぞれの母集団の比率を$p_1$,$p_2$とした場合を考えます。



比率 $\hat{p}_1 (=\frac{x_1}{n_1})$が従う確率分布は二項分布です。  
また、$n$が大きいときには正規分布で近似することができます。

平均$μ_1=p_1$   
標準偏差$σ_1=\sqrt{\frac{p_1(1-p_1)}{n_1}}$

比率 $\hat{p}_2 (=\frac{x_2}{n_2})$が従う確率分布は二項分布です。  
また、$n$が大きいときには正規分布で近似することができます。

平均$μ_2=p_2$  
標準偏差$σ_2=\sqrt{\frac{p_2(1-p_2)}{n_2}}$

今回求めたいのは $\hat{p}_1-\hat{p}_2（\frac{x_1}{n_1}-\frac{x_2}{n_2}）$ が従う確率分布です。

どのような確率分布に従うのでしょうか？

比率の差の標本分布の統計量は、それぞれの比率の標本分布の  
**平均の差**と**分散の和**になります。

（経緯については[こちら](https://bit.ly/3EcZXeh)が参考になると思います）

今回は帰無仮説が正しいと仮定しているので、$p_1=p_2$ となります。

ここで、母比率を $p_1=p_2=p$ とします。

母比率$p$とサンプルサイズ$n_1,n_2$を用いて、$z$値を導出します。  
（上方・下方の信頼限界を求める方法もあります）

標準偏差 $σ=\sqrt{\frac{p_1(1-p_1)}{n_1}+\frac{p_2(1-p_2)}{n_2}} = \sqrt{p(1-p)(\frac{1}{n_1}+\frac{1}{n_2})}$



よって 
$$
z=\frac{\frac{x_1}{n_1}-\frac{x_2}{n_2}}{\sqrt{p(1-p)(\frac{1}{n_1}+\frac{1}{n_2})}}
$$

上の$z$の式には母集団の比率$p$が入っています。  
当然未知の値なので、標本比率（$\hat{p}$とします）を代わりの値として代入します。

今回は$8000$世帯中$1540$世帯が視聴していたので、  
$1540/8000$ を代入しましょう。

よって 
$$
z=\frac{\frac{x_1}{n_1}-\frac{x_2}{n_2}}{\sqrt{\hat{p}(1-\hat{p})(\frac{1}{n_1}+\frac{1}{n_2})}}
$$

上の式に従って$z$値を導出します。

#### ③ 帰無仮説を棄却できるかどうかを確認する

有意水準を$5$%に設定します。

②で導出した$z$値と、有意水準に基づく$z$値を比較し、  
($p$**値を求めて**) 棄却が必要かどうか決めます。  


### Pythonによる仮説検定

では母比率の差の検定を行っていきましょう。

In [21]:
print("H0 : p1＝p2 [帰無仮説]")  # 関東地区と関西地区の視聴率は等しい
print("H1 : p1≠p2 [対⽴仮説]")  # 関東地区と関西地区の視聴率は等しくない

H0 : p1＝p2 [帰無仮説]
H1 : p1≠p2 [対⽴仮説]


In [22]:
# ([標本の中で観察できた数],[標本の大きさ],[検定方法（今回は両側検定）])
nk , nkp = proportions_ztest([1000,540],[5000,3000],alternative="two-sided")

nk = float(Decimal(str(nk)).quantize(Decimal('0.001'),rounding=ROUND_HALF_UP))
print(f"検定統計量のz値:{nk}")
nkp = float(Decimal(str(nkp*100)).quantize(Decimal('0.001'),rounding=ROUND_HALF_UP))
print(f"p値：{nkp}%")

検定統計量のz値:2.197
p値：2.805%


* $z$値が$1.96$より大きくなった
* $p$値が$5$%より小さくなった

この二点から、帰無仮説$H_0$を棄却することができます。  
よって、対立仮説$H_1$を採択します。

In [23]:
print("したがって、関東地区と関西地区とで視聴率に差がある")

したがって、関東地区と関西地区とで視聴率に差がある
